# Introdução à Inteligência Artificial 2025/2026

## Projeto 2: A Procura da Lagarta

### Entrega: 20 de Outubro às 23:59

<left> <img src="figures/lagarta_P2_inicio.png" width="400" /> </center>

## Introdução e Objetivos

Recordamos o jogo da lagarta, cujo objetivo é fazer a lagarta chegar à maçã. No projeto anterior pretendia-se a formulação do problema. Neste projeto pretende-se duas coisas:

* desenvolver uma **nova heurística** que possa ser usada por algoritmos de procura informada;
* desenvolver um **novo algoritmo de procura** que melhore o desempenho da procura cega.

Ambos serão descritos a seguir, mas primeiro fazemos um resumo do Mundo da Lagarta do projeto anterior e das diferenças introduzidas neste projeto.

Vão precisar dos módulos seguintes (distribuídos juntamente com o enunciado):
* `searchPlus_better.py`
* `utils.py`

## Recordar o Mundo da Lagarta

Podem recordar o Mundo da Lagarta (o projeto anterior) no enunciado respetivo, mas fica aqui uma breve descrição:

O Mundo da Lagarta é um mundo simples onde existem apenas paredes e uma maçã, para além da própria lagarta. As paredes rodeiam completamente o mundo (que pode ser quadrado ou retangular) e podem existir também no seu interior. A maçã pode estar num sítio qualquer, apoiada em cima de uma parede ou a levitar, mas a lagarta está sujeita à força de gravidade:

* a posição da lagarta é definida pela posição da sua cabeça;
* a lagarta só pode mover-se ortogonalmente (não na diagonal) para casas imediatamente adjacentes à casa onde está posicionada;
* a cada movimento que faz, a lagarta cresce: a cabeça move-se mas fica corpo onde estava a cabeça;
* a lagarta só pode mover-se para casas livres (não pode atravessar paredes nem o seu próprio corpo) ou para a casa onde está a maçã;
* se a cabeça da lagarta não estiver apoiada (a célula imediatamente abaixo está livre) a única ação possível é para baixo (B);
* a ação de levantar a cabeça para cima (C) só é possível se o nível máximo de esforço ainda não tiver sido atingido (nível máximo = 3), e esta ação aumenta o nível de esforço em 1 unidade;
* se a lagarta estiver em esforço (nível de esforço > 0) só pode ir para a direita (D) ou esquerda (E) se isso a colocar numa casa com apoio;
* independentemente do nível de esforço anterior, se a lagarta se move para uma casa com apoio o seu nível de esforço passa a ser 0;
* uma casa com apoio é uma casa imediatamente acima de uma parede ou do corpo da lagarta.

## O que mudou

Neste projeto, a lagarta apresenta apenas uma diferença em relação ao projeto anterior. Agora as suas ações têm custos diferenciados. Mais precisamente:

* ir para baixo tem custo 1;
* ir para a esquerda ou para a direita tem custo 2;
* ir para cima tem custo 3.

Sendo assim, agora o objetivo da lagarta é chegar à maçã com o **menor custo possível**.


## Formulação do Mundo da Lagarta

A seguir apresentamos a implementação da classe `MundoLagarta` para a modelização do problema como um grafo de estados, o qual utilizámos como referência para a realização dos testes do projeto anterior. Repare-se que, não sendo necessário no projeto anterior, agora definimos o método `path_cost` de acordo com os custos diferenciados referidos acima. Implementamos também a classe `EstadoLagarta`, definindo os métodos `__hash__` e `__lt__` que são necessários apenas para podermos utilizar métodos de procura cuja fronteira é uma `PriorityQueue`.

Incluímos também já uma heurística, `h_dist`, que se limita a reconhecer se um estado é final e a calcular a distância de Manhattan entre a posição da (cabeça da) lagarta e a maçã (retorna 0 na presença de um estado final e a referida distância em todos os outros casos).

In [128]:
from numbers import Number
from searchPlus_better import *
import copy

line1 = "= = = = = = =\n"
line2 = "= x . . . . =\n"
line3 = "= . . . = . =\n"
line4 = "= . . . = . =\n"
line5 = "= = = . = . =\n"
line6 = "= @ . . . . =\n"
line7 = "= = = = = = =\n"
grelha = line1 + line2 + line3 + line4 + line5 + line6 + line7

def manhattan(p,q):
    (x1,y1) = p
    (x2,y2) = q
    return abs(x1-x2) + abs(y1-y2)

class EstadoLagarta(dict): 

    def __hash__(self):
        body_sorted = tuple(sorted(self['body']))
        head = self['head']
        effort = self['effort']
        return hash((body_sorted, head, effort))
    
    def __lt__(self,other):
        """Um estado é sempre menor do que qualquer outro, para desempate na fila de prioridades"""
        return True

class MundoLagarta(Problem):

    def process_txt(self, grid):
        data = {'walls': set(), 'body': set()}
        lines = grid.split('\n')
        # atenção que a primeira linha é a de cima, mas queremos que essa seja
        # a linha y=len(lines)-1 e que a de baixo seja a linha y=0
        # então vamos inverter:
        lines.reverse()
        y = 0
        for row in lines[1:]: # ignora primeira linha, vazia (era a última linha)
            x = 0
            for col in row:
                if col == '=':
                    data['walls'].add((x,y))
                elif col == 'x':
                    data['apple'] = (x,y)
                elif col == '@':
                    data['head'] = (x,y)
                elif col == 'o':
                    data['body'].add((x,y))
                if col != " ":
                    x += 1
            y += 1
        data['dim'] = ((len(lines[-1])+1)//2,len(lines)-1)
        return data
    
    directions = {"E":(-1, 0), "D":(+1, 0), "C":(0, +1), "B":(0, -1)}  # ortogonals

    def __init__(self, MundoInicial=grelha, esforco_max=3):
        initialStatus = self.process_txt(MundoInicial) # process txt and convert to a dictionary
        self.initial=EstadoLagarta()
        self.initial['head']=initialStatus['head']
        self.initial['body']=initialStatus['body']
        self.initial['effort']=0
        self.goal = initialStatus['apple'] # goal position
        self.walls = initialStatus['walls'] # walls positions
        self.dim = initialStatus['dim'] # maze dimension (do not need to be squared)
        self.emax = esforco_max
        
    def actions (self, state):
        x, y = state['head'] # head position
        b = state['body'] # body
        e = state['effort'] # effort
        action_list = []

        # if lagarta has an empty space below the head, the only possible action is down:
        below_pos = (x,y-1)
        if below_pos not in self.walls and below_pos not in b:
            action_list = ['B'] # só pode ir para baixo
            return action_list
        
        # lagarta can go up if above cell is free and current effort is < max effort: 
        new_pos = (x+self.directions['C'][0],y+self.directions['C'][1])
        if new_pos not in self.walls and new_pos not in b and e < 3:
            action_list.append('C')
        
        # going left is possible if:
        # - left cell is free and supported (below there is wall or body)
        #   or
        # - left cell is free and unsupported and lagarta is not in effort
        new_pos = (x+self.directions['E'][0],y+self.directions['E'][1])
        if new_pos not in self.walls and new_pos not in b:
            if e == 0:
                action_list.append('E')
            else:
                below_pos = (new_pos[0],new_pos[1]-1)
                if below_pos in self.walls or below_pos in b:
                    action_list.append('E')
        
        # going right is possible if:
        # (same thing, but right)
        new_pos = (x+self.directions['D'][0],y+self.directions['D'][1])
        if new_pos not in self.walls and new_pos not in b:
            if e == 0:
                action_list.append('D')
            else:
                below_pos = (new_pos[0],new_pos[1]-1)
                if below_pos in self.walls or below_pos in b:
                    action_list.append('D')
            
        return sorted(action_list)
    
    def result (self, state, action):
        clone=copy.deepcopy(state)
        x, y = clone['head']
        b = clone['body']
        e = clone['effort']
        # determine new position of head:
        new_pos = (x+self.directions[action][0], y+self.directions[action][1])
        clone['head'] = new_pos
        # determine whether effort is increased:
        if action == 'C':
            clone['effort'] += 1
        # determine whether effort goes back to 0:
        if (new_pos[0],new_pos[1]-1) in self.walls or (new_pos[0],new_pos[1]-1) in b:
            clone['effort'] = 0
        # body fills previous head position:
        clone['body'].add((x,y))
        return clone
    
    def goal_test (self, state):
        return state['head'] == self.goal

    def path_cost(self, c, state1, action, state2):
        if action == 'C':
            cost_action = 3
        elif action == 'B':
            cost_action = 1
        else:
            cost_action = 2
        return c + cost_action
    
    def display (self, state):
        """Devolve a grelha em modo txt"""
        output = ""
        for y in range(self.dim[1]-1,-1,-1):
        # atenção, invertemos aqui a ordem dos y, pois para display começamos por cima    
            for x in range(self.dim[0]):
                if state['head'] == (x,y):
                    ch = "@"
                elif self.goal == (x,y):
                    ch = "x"
                elif (x,y) in self.walls:
                    ch = "="
                elif (x,y) in state['body']:
                    ch = "o"
                else:
                    ch = "."
                output += ch + " "
            output += "\n"
        return output
    
    def executa(self, state, actions_list, verbose=False):
        """Executa uma sequência de acções a partir do estado devolvendo o triplo formado pelo estado, 
        pelo custo acumulado e pelo booleano que indica se o objectivo foi ou não atingido. Se o objectivo 
        for atingido antes da sequência ser atingida, devolve-se o estado e o custo corrente.
        Há o modo verboso e o não verboso, por defeito."""
        cost = 0
        for a in actions_list:
            seg = self.result(state,a)
            cost = self.path_cost(cost,state,a,seg)
            state = seg
            obj = self.goal_test(state)
            if verbose:
                print('Ação:', a)
                print(self.display(state),end='')
                print('Custo Total:',cost)
                print('Esforço:',state['effort'])
                print('Atingido o objectivo?', obj)
                print()
            if obj:
                break
        return (state, cost, obj)

    def h_dist(self, node):
        """ """
        clone=copy.deepcopy(node.state)
        ## Satisfaz objectivo?
        if self.goal_test(clone):
            return 0
        distancia_lagarta_maca = manhattan(clone['head'],self.goal)
        return distancia_lagarta_maca













    def h_dist_costs(self, node):
        clone=copy.deepcopy(node.state)
        ## Satisfaz objectivo?
        if self.goal_test(clone):
            return 0
        x1, y1 = clone['head']
        x2, y2 = self.goal
        yDif = y1 - y2
        hCost = 2 * abs(x1 - x2)
        yCost = 3 * abs(yDif) if yDif <= 0 else abs(yDif)
        return hCost + yCost






# def join_lists268(n, first=[], second=[])->list[Node]:
#     if (len(first) + len(second) <= n):
#         first.extend(second)
#         return first
#     offset = len(first) - len(second) % n
#     first.extend(second)
#     result = first
#     result[len(result) - n:]
#     result[n - offset:].extend(result[:n - offset])
#     return result

# def join_lists47(n, first=[], second=[])->list[Node]:
#     if (len(first) + len(second) <= n):
#         first.extend(second)
#         return first
#     result = [None] * n
#     result[:len(first)] = first
#     offset = len(first)
#     for el in second:
#         result[offset % n] = el 
#     return result # Circular buffer



# # If there is space, new nodes are added to front of list, else they are discarded
# def join_lists224(n, first, second)->list[Node]:
#     if (len(first) + len(second) <= n):
#         first.extend(second)
#         return first
#     s = second[:n]
#     s.extend(first)
#     return s[:n]



def get_best_node(boundary: list[tuple[Node, float]]):
    nodes, novelties = zip(*boundary)
    positions = [(-node.state['head'][0], -node.state['head'][1]) for node in nodes]
    xs = list(zip(novelties, positions))
    return boundary[xs.index(max(xs))]



def calc_novelties(nodes: list) -> list[tuple[Node, float]]:
    print(nodes)
    if (len(nodes) <= 1):
        return list(zip(nodes, [0.0]))
    distances = [[0.0]] * len(nodes)
    novelties = []
    for i in range(len(nodes)):
        this = nodes[i]
        others = [nodes[j] for j in range(len(nodes)) if i != j]
        distances[i] = [manhattan(this.state['head'], other.state['head']) for other in others]
    novelties = list(map(lambda ls: sum(ls) / len(ls), distances))
    print(distances)
    print(novelties)
    return list(zip(nodes, novelties))



# Nodes are not overriden, if there is space they are added, if not they arent.
def join_lists(n, first, second):
    first.extend(second)
    return first[:n]




def graph_search_count_novelty(problem, N: int, frontier: list):
    """
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty list.
    If two paths reach a state, only use the first one.
    The node to expand should be the one with highest novelty.
    In case of ties, second criterium is the position of the head, lowest first.
    The maximum number of nodes in the frontier is N.
    The novelty of a node is measured as the average (Manhattan) distance
    between the node state and the states of all other nodes in the frontier.
    """
    expandidos=0
    frontier.append(Node(problem.initial))
    boundary = list(zip(frontier, [0.0]))
    explored = []
    while True:
        if (len(boundary) == 0):
            break
        # chooses node to expand, if it is the goal return.
        node, novelty = get_best_node(boundary)
        expandidos += 1
        if problem.goal_test(node.state):
            return (node, expandidos)
        #
        explored.append(node.state)
        boundaryNodes, _ = zip(*boundary)
        boundaryNodes = list(boundaryNodes)
        extension = [child for child in node.expand(problem) if child.state not in explored and child not in boundaryNodes]

        boundary.remove((node, novelty))
        boundaryNodes.remove(node)

        nodes = join_lists(N, boundaryNodes, extension)
        boundary = calc_novelties(nodes)

    return (None,expandidos)



In [95]:
novs = [5.6, 5.6, 5.6]
pos = [(5, 6), (4,3), (3,4)]
positions = [(-p[0], -p[1]) for p in pos]
xs = list(zip(novs, positions))

print(xs.index(max(xs)))

2


In [129]:
 	

# try:
line1 = "= = = = = = = = = = = = = = = = = = =\n"
line2 = "= . . . . . . . x . . . . . . . . . =\n"
line3 = "= . . . = = . = = = . . . . . . . . =\n"
line4 = "= . . . . . . . . . . . . . . . . . =\n"
line5 = "= . = = = = = = = = = = = = . . . . =\n"
line6 = "= . . . . . . . . . . . . . . . . . =\n"
line7 = "= . . . . = = = = = = = . . . . . . =\n"
line8 = "= . . . . . . . @ . . . . . . . . . =\n"
line9 = "= = = = = = = = = = = = = = = = = = =\n"
grelha2 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9
p = MundoLagarta(grelha2)
resultado1, n_explored1 = astar_search_plus_count(p,p.h_dist_costs)
resultado2, n_explored2 = graph_search_count_novelty(p,3,[])
if resultado1 and resultado2:
    print(n_explored1>n_explored2)
else:
    print("Sem solução!")
# except Exception as e:
#     print(repr(e))

[<Node {'head': (9, 1), 'body': {(8, 1)}, 'effort': 0}>, <Node {'head': (7, 1), 'body': {(8, 1)}, 'effort': 0}>]
[[2], [2]]
[2.0, 2.0]
[<Node {'head': (9, 1), 'body': {(8, 1)}, 'effort': 0}>, <Node {'head': (6, 1), 'body': {(7, 1), (8, 1)}, 'effort': 0}>]
[[3], [3]]
[3.0, 3.0]
[<Node {'head': (9, 1), 'body': {(8, 1)}, 'effort': 0}>, <Node {'head': (5, 1), 'body': {(6, 1), (7, 1), (8, 1)}, 'effort': 0}>]
[[4], [4]]
[4.0, 4.0]
[<Node {'head': (9, 1), 'body': {(8, 1)}, 'effort': 0}>, <Node {'head': (4, 1), 'body': {(6, 1), (7, 1), (5, 1), (8, 1)}, 'effort': 0}>]
[[5], [5]]
[5.0, 5.0]
[<Node {'head': (9, 1), 'body': {(8, 1)}, 'effort': 0}>, <Node {'head': (4, 2), 'body': {(7, 1), (8, 1), (6, 1), (5, 1), (4, 1)}, 'effort': 1}>, <Node {'head': (3, 1), 'body': {(7, 1), (8, 1), (6, 1), (5, 1), (4, 1)}, 'effort': 0}>]
[[6, 6], [6, 2], [6, 2]]
[6.0, 4.0, 4.0]
[<Node {'head': (4, 2), 'body': {(7, 1), (8, 1), (6, 1), (5, 1), (4, 1)}, 'effort': 1}>, <Node {'head': (3, 1), 'body': {(7, 1), (8, 1), (

In [130]:
line1 = "= = = = = = = = = = = = = = = = = = =\n"
line2 = "= . . . . . . . x . . . . . . . . . =\n"
line3 = "= . . . = = . = = = . . . . . . . . =\n"
line4 = "= . . . . . . . . . . . . . . . . . =\n"
line5 = "= . = = = = = = = = = = = = . . . . =\n"
line6 = "= . . . . . . . . . . . . . . . . . =\n"
line7 = "= . . . . = = = = = = = . . . . . . =\n"
line8 = "= . . . . . . . @ . . . . . . . . . =\n"
line9 = "= = = = = = = = = = = = = = = = = = =\n"
grelha2 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9
p = MundoLagarta(grelha2)
resultado1, n_explored1 = graph_search_count_novelty(p,10,[])
resultado2, n_explored2 = graph_search_count_novelty(p,20,[])
if resultado1 and resultado2:
    print(n_explored1<n_explored2) #expected True
else:
    print("Sem solução!")

[<Node {'head': (9, 1), 'body': {(8, 1)}, 'effort': 0}>, <Node {'head': (7, 1), 'body': {(8, 1)}, 'effort': 0}>]
[[2], [2]]
[2.0, 2.0]
[<Node {'head': (9, 1), 'body': {(8, 1)}, 'effort': 0}>, <Node {'head': (6, 1), 'body': {(7, 1), (8, 1)}, 'effort': 0}>]
[[3], [3]]
[3.0, 3.0]
[<Node {'head': (9, 1), 'body': {(8, 1)}, 'effort': 0}>, <Node {'head': (5, 1), 'body': {(6, 1), (7, 1), (8, 1)}, 'effort': 0}>]
[[4], [4]]
[4.0, 4.0]
[<Node {'head': (9, 1), 'body': {(8, 1)}, 'effort': 0}>, <Node {'head': (4, 1), 'body': {(6, 1), (7, 1), (5, 1), (8, 1)}, 'effort': 0}>]
[[5], [5]]
[5.0, 5.0]
[<Node {'head': (9, 1), 'body': {(8, 1)}, 'effort': 0}>, <Node {'head': (4, 2), 'body': {(7, 1), (8, 1), (6, 1), (5, 1), (4, 1)}, 'effort': 1}>, <Node {'head': (3, 1), 'body': {(7, 1), (8, 1), (6, 1), (5, 1), (4, 1)}, 'effort': 0}>]
[[6, 6], [6, 2], [6, 2]]
[6.0, 4.0, 4.0]
[<Node {'head': (4, 2), 'body': {(7, 1), (8, 1), (6, 1), (5, 1), (4, 1)}, 'effort': 1}>, <Node {'head': (3, 1), 'body': {(7, 1), (8, 1), (

In [126]:
line1 = "= = = = = = = = = = = = = = = = = = =\n"
line2 = "= . . . . . . . x . . . . . = . . . =\n"
line3 = "= . . . = = . = = = . . . . = . . . =\n"
line4 = "= . . . . . . . . . . . . . = . . . =\n"
line5 = "= . = = = = = = = = = = = = = . . . =\n"
line6 = "= . . . . . . . . . . . . . = . . . =\n"
line7 = "= . . . . = = = = = = = . . = . . . =\n"
line8 = "= . . . . . . . @ . . . . . . . . . =\n"
line9 = "= = = = = = = = = = = = = = = = = = =\n"
grelha2 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9
p = MundoLagarta(grelha2)
resultado1, n_explored1 = graph_search_count_novelty(p,10,[])
resultado2, n_explored2 = graph_search_count_novelty(p,20,[])
if resultado1 and resultado2:
    print(n_explored1) #expected 172
    print(n_explored2)
else:
    print("Sem solução!")

122
268


In [127]:
try:
    line1 = "= = = = = = = = = = = = = = = = = = =\n"
    line2 = "= . . . . . . . x . . . . . . . . . =\n"
    line3 = "= . . . = = . = = = . . . . . . . . =\n"
    line4 = "= . . . . . . . . . . . . . . . . . =\n"
    line5 = "= . = = = = = = = = = = = = . . . . =\n"
    line6 = "= . . . . . . . . . . . . . . . . . =\n"
    line7 = "= . . . . = = = = = = = . . . . . . =\n"
    line8 = "= . . . . . . . @ . . . . . . . . . =\n"
    line9 = "= = = = = = = = = = = = = = = = = = =\n"
    grelha2 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9
    p = MundoLagarta(grelha2)
    resultado,_ = graph_search_count_novelty(p,3,[])
    if resultado:
        print("Solução Novelty c/ N=3 com custo", str(resultado.path_cost)+":")
        print(resultado.solution())
    else:
        print("Sem solução!")
except Exception as e:
    print(repr(e))

Solução Novelty c/ N=3 com custo 90:
['E', 'E', 'E', 'E', 'C', 'C', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'B', 'B', 'D', 'D', 'C', 'E', 'C', 'D', 'C', 'C', 'E', 'C', 'D', 'C', 'E', 'E', 'B', 'B', 'E', 'E', 'E', 'E', 'E', 'E', 'C', 'C', 'D', 'D']


In [94]:
print("Solução Novelty c/ N=3 com custo 90:\n['E', 'E', 'E', 'E', 'E', 'E', 'C', 'D', 'D', 'C', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'B', 'B', 'D', 'D', 'D', 'D', 'D', 'C', 'E', 'E', 'E', 'E', 'C', 'D', 'C', 'C', 'E', 'E', 'E', 'E', 'C', 'C', 'E', 'E']")

Solução Novelty c/ N=3 com custo 90:
['E', 'E', 'E', 'E', 'E', 'E', 'C', 'D', 'D', 'C', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'B', 'B', 'D', 'D', 'D', 'D', 'D', 'C', 'E', 'E', 'E', 'E', 'C', 'D', 'C', 'C', 'E', 'E', 'E', 'E', 'C', 'C', 'E', 'E']


Solução Novelty c/ N=3 com custo 50:↩
['E', 'E', 'E', 'E', 'E', 'E', 'E', 'C', 'D', 'C', 'E', 'C', 'C', 'D', 'D', 'D', 'D', 'D', 'C', 'C', 'D', 'D']

Solução Novelty c/ N=3 com custo 90:↩
['E', 'E', 'E', 'E', 'E', 'E', 'C', 'D', 'D', 'C', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'B', 'B', 'D', 'D', 'D', 'D', 'D', 'C', 'E', 'E', 'E', 'E', 'C', 'D', 'C', 'C', 'E', 'E', 'E', 'E', 'C', 'C', 'E', 'E']

## Nova heurística

Olhando para o código acima, vê-se no final a definição por completar de uma nova heurística chamada `h_dist_costs`, cuja implementação devem fazer. **Não mexam** em mais nada da classe!

Esta heurística deve calcular uma distância de Manhattan especial, que considera os custos das ações como se fossem distâncias. Tal como a heurística `h_dist`, deve considerar uma versão relaxada do problema em que não há paredes no interior do mundo.

Seguem-se dois exemplos de mundos e os respetivos valores heurísticos que seriam devolvidos pela nova heurística `h_dist_costs`.

**1)** Se estivermos neste mundo:
```
= = = = = = = = = = = = = = = = = = =
= . . . . . . . . . . . . . . . . . =
= . . . . . . . . . . . . . . . . . =
= . . . . . . . . . . . . = . . . . =
= . . . . . . o o o . @ . = . . . . =
= . . . . . . o = o . o = = . . . . =
= . . o o o o o = o o o = = . . x . =
= = = = = = = = = = = = = = = = = = =
```
A heurística `h_dist_costs` deve devolver 12 (a lagarta tem de fazer cinco movimentos para a direita, cada um custando 2, e dois movimentos para baixo, cada um custando 1), ignorando as paredes no interior do mundo.


**2)** Se estivermos neste mundo:
```
= = =
= x =
= . =
= . =
= . =
= . =
= @ =
= = =
```
A heurística `h_dist_costs` deve devolver 15 (cinco movimentos para cima, cada um custando 3), mesmo que seja impossível chegar à maçã.


## Novo algoritmo de procura

Devem implementar um novo algoritmo de procura cega baseado em *Novelty Search*. No nosso problema, a inovação (*novelty*) será medida em termos de distâncias. Vamos tomar a liberdade de dizer que a localização de um estado é a posição (coordenadas) da lagarta nesse estado. Sendo assim, um estado será mais inovador se estiver mais distante dos outros, e será menos inovador se estiver mais próximo dos outros.

Especificando ainda melhor, no novo algoritmo de procura vamos precisar de atribuir um valor de *novelty* a cada um dos estados/nós na fronteira, e esse valor será a média das distâncias de Manhattan a todos os outros estados/nós na fronteira. O primeiro a sair da fronteira será aquele com **maior valor de *novelty***. Em caso de empate, escolhe-se aquele com **localização *menor*** (considerando uma ordenação ascendente pelas coordenadas da posição da lagarta). Exemplificando a ordenação das coordenadas: (8,2) < (9,2) < (9,3).

Atenção, esta é uma procura cega em que **a fronteira é uma lista** (não é FIFOQueue, não é PriorityQueue, etc). 

Mais ainda, **a fronteira tem tamanho limitado**, só lá podem estar N nós de cada vez, e os que não cabem são esquecidos, não serão expandidos. Claramente, isto faz com que este algoritmo não garanta encontrar uma solução.

Implementem este algoritmo completando o código abaixo. Sugerimos que se baseiem no código do `graph_search_count` (procura em grafo que conta os nós expandidos) disponível em `searchPlus_better.py`. Naturalmente, podem acrescentar quaisquer funções e outros elementos auxiliares que considerem necessários.

In [2]:
def graph_search_count_novelty(problem, N, frontier):
    """Search through the successors of a problem to find a goal.
    The argument frontier should be an empty list.
    If two paths reach a state, only use the first one.
    The node to expand should be the one with highest novelty.
    In case of ties, second criterium is the position of the head, lowest first.
    The maximum number of nodes in the frontier is N.
    The novelty of a node is measured as the average (Manhattan) distance
    between the node state and the states of all other nodes in the frontier."""
    expandidos=0
    pass
    return (None,expandidos)

## Submissão

#### Quizz

Cada aluno deve completar a implementação da heurística `h_dist_costs` e do algoritmo de procura `graph_search_count_novelty` e testar ambos no link do quizz **Projeto 2** que está na página da disciplina, introduzindo aí o seu código. Pode-se submeter e avaliar o código várias vezes, sendo a submissão com melhor nota a que será considerada. Este projeto deve ser realizado **individualmente**.

O quizz é constituído por uma única pergunta. A avaliação será feita através de um conjunto de testes automáticos visíveis e outros invisíveis, valendo um total de 2,15 valores da avaliação da Unidade Curricular. Metade dos testes avaliam a nova heurística enquanto os restantes avaliam o novo algoritmo de procura. Os testes visíveis valem 6 em 20 e os testes invisíveis valem 14 em 20.

#### Ficheiro Pyhton

Simultaneamente, é necessário submeter o script Python que contém todo o código submetido no quizz. **Sem esta submissão o trabalho não poderá ser avaliado**, independentemente do resultado obtido no Moodle. Esse ficheiro deve chamar-se **ProcuraLagarta_IIA_25_26_alunoXXXXX.py**, em que substituem XXXXX pelo número do aluno.

## **Bom trabalho!**
<left> <img src="figures/lagarta_P2_fim.png" width="300" /> </center>

(imagens meramente ilustrativas retiradas e/ou adaptadas de: https://jayisgames.com/review/lime-rick.php)
